# Jet Clustering

This workflow is for use with the jet samples, that contain both `ClusterTree` and `EventTree` (provided by the `MLTree` utility). This **cannot** handle data where the `EventTree` does not exist, because that contains info on piecing the clusters together into events*, and the baseline jet clustering.

\* This pieceing together can be accomplished in workflows like `EventReconstructionPion.ipynb` but it's rather complex.

#### TODO:

- jet clustering
- comparison of jets

#### 1) Setup

First, let's import a bunch of packages we know we'll need right off-the-bat.

Note that as we've set up our environment with `conda`, our `ROOT` installation has all the bells and whistles. This includes the `pythia8` library and its associated `ROOT` wrapper, `TPythia8`. We can optionally use this for jet-clustering, as it comes `fj-core`.
Alternatively we could use the Pythonic interface for `fastjet` or [pyjet](https://github.com/scikit-hep/pyjet), but the latter requires linking an external fastjet build for speed and this doesn't seem to work when following their documentation.

In [ ]:
# Imports - generic stuff

import numpy as np
import ROOT as rt
import uproot as ur
import sys, os, glob, uuid
import subprocess as sub
from pathlib import Path

path_prefix = '/workspace/LCStudies/'
if(path_prefix not in sys.path): sys.path.append(path_prefix)
from util import ml_util as mu # for passing calo images to regression networks
from util import qol_util as qu # for progress bar

In [ ]:
# Imports and setup for TensorFlow and Keras.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # disable some of the tensorflow info printouts, only display errors
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

ngpu = 1
gpu_list = ["/gpu:"+str(i) for i in range(ngpu)]
strategy = tf.distribute.MirroredStrategy(devices=gpu_list)
ngpu = strategy.num_replicas_in_sync
print ('Number of devices: {}'.format(ngpu))

# Dictionary for storing all our neural network models that will be evaluated
network_models = {}

In [ ]:
# setup paths
data_dir = path_prefix + 'data/jet'
classification_dir = path_prefix + 'classifier/Models'
regression_dir = path_prefix + 'regression/Models'
fj_dir = path_prefix + '/setup/fastjet/fastjet-install/lib/python3.8/site-packages'

In [ ]:
# ----- Calorimeter meta-data -----
layers = ["EMB1", "EMB2", "EMB3", "TileBar0", "TileBar1", "TileBar2"]
nlayers = len(layers)
cell_size_phi = [0.098, 0.0245, 0.0245, 0.1, 0.1, 0.1]
cell_size_eta = [0.0031, 0.025, 0.05, 0.1, 0.1, 0.2]
len_phi = [4, 16, 16, 4, 4, 4]
len_eta = [128, 16, 8, 4, 4, 2]
assert(len(len_phi) == nlayers)
assert(len(len_eta) == nlayers)
meta_data = {
    layers[i]:{
        'cell_size':(cell_size_eta[i],cell_size_phi[i]),
        'dimensions':(len_eta[i],len_phi[i])
    }
    for i in range(nlayers)
}

In [ ]:
# flat classifiers
print('Loading flat classification models... ')
flat_model_files = glob.glob(classification_dir + '/flat/' + '*.h5')
flat_model_files.sort()
flat_model_names = []
for model in flat_model_files:
    model_name = model.split('model_')[-1].split('_flat')[0]
    print('\tLoading ' + model_name + '... ',end='')
    flat_model_names.append(model_name)
    network_models[model_name] = tf.keras.models.load_model(model)
    print('Done.')

# combo classifier
print('Loading simple combo classification model... ',end='')
combo_model_file = classification_dir + '/simple/' + 'model_simple_do20.h5'
network_models['combo'] = tf.keras.models.load_model(combo_model_file)
print('Done.')

# energy regression networks
print('Loading charged-pion energy regression model... ',end='')
charged_energy_model_file = regression_dir + '/' + 'all_charged.h5'
network_models['e_charged'] = tf.keras.models.load_model(charged_energy_model_file)
print('Done.')

print('Loading neutral-pion energy regression model... ',end='')
neutral_energy_model_file = regression_dir + '/' + 'all_neutral.h5'
network_models['e_neutral'] = tf.keras.models.load_model(neutral_energy_model_file)
print('Done.')

Now we make a "local" copy of the jet data. We will only copy over certain branches, and we will skip any files that don't contain an `eventTree` in them.

In [ ]:
data_filenames = glob.glob(data_dir + '/' + '*.root')

# debugging - lets us use a single file to speed stuff up a lot.
#data_filenames = [data_dir + '/' + 'user.angerami.21685345.OutputStream._000062.root']

# our "local" data dir, where we create modified data files
jet_data_dir = path_prefix + 'jets/data'
Path(jet_data_dir).mkdir(parents=True, exist_ok=True)

# Get the original data.
files = {name:rt.TFile(name,'READ') for name in data_filenames}

# Some data files might be missing an EventTree.
# For now, we will skip these because our methods count on an existing EventTree.
delete_keys = []
for key, val in files.items():
    file_keys = [x.GetName() for x in val.GetListOfKeys()]
    if('ClusterTree' not in file_keys or 'EventTree' not in file_keys):
        delete_keys.append(key)

for key in delete_keys: 
    print('Ignoring file:',key,'(no EventTree/ClusterTree found).')
    del files[key]

if(path_prefix not in sys.path): sys.path.append(path_prefix)
from  util import qol_util as qu # for progress bar

# now we make a local copy of the files in the jet_data_dir, keeping only certain branches
active_branches = {}
active_branches['cluster'] = [
    'runNumber',
    'eventNumber',
    'truthE',
    'truthPt',
    'truthEta',
    'truthPhi',
    'clusterIndex',
    'nCluster',
    'clusterE',
    'clusterECalib',
    'clusterPt',
    'clusterEta',
    'clusterPhi',
    'cluster_nCells',
    'cluster_ENG_CALIB_TOT',
    'EMB1',
    'EMB2',
    'EMB3',
    'TileBar0',
    'TileBar1',
    'TileBar2'
]
active_branches['event'] = [
    'runNumber',
    'eventNumber',
    'lumiBlock',
    'NPV',
    'nTruthPart',
    'clusterCount',
    'nCluster',
    'clusterE',
    'clusterPt',
    'clusterEta',
    'clusterPhi',
    'AntiKt4EMTopoJetsPt',
    'AntiKt4EMTopoJetsEta',
    'AntiKt4EMTopoJetsPhi',
    'AntiKt4EMTopoJetsE',
    'AntiKt4LCTopoJetsPt',
    'AntiKt4LCTopoJetsEta',
    'AntiKt4LCTopoJetsPhi',
    'AntiKt4LCTopoJetsE',
    'AntiKt4TruthJetsPt',
    'AntiKt4TruthJetsEta',
    'AntiKt4TruthJetsPhi',
    'AntiKt4TruthJetsE'
]

tree_names = {'cluster':'ClusterTree','event':'EventTree'}
data_filenames = []

l = len(files.keys())
i = 0
qu.printProgressBarColor(i, l, prefix='Copying data files:', suffix='Complete', length=50)

for path, tfile in files.items():
    filename_new = jet_data_dir + '/' + path.split('/')[-1]
    old_trees = {x:tfile.Get(tree_names[x]) for x in tree_names.keys()}
    
    for key, tree in old_trees.items():
        tree.SetBranchStatus('*',0)
        for bname in active_branches[key]: tree.SetBranchStatus(bname,1)
    
    tfile_new = rt.TFile(filename_new,'RECREATE')
    new_trees = {x:old_trees[x].CloneTree() for x in old_trees.keys()}
    tfile_new.Write()
    data_filenames.append(filename_new)
    i += 1
    qu.printProgressBarColor(i, l, prefix='Copying data files:', suffix='Complete', length=50)
    del old_trees
    del new_trees

In [ ]:
# Access the files & trees with uproot
tree_names = {'cluster':'ClusterTree','event':'EventTree'}
ur_trees = {file:{tree_key:ur.open(file)[tree_name] for tree_key,tree_name in tree_names.items()} for file in data_filenames}

Now we will loop over our data files. This isn't the most notebook-esque code, but it should avoid "out of memory" issues: As we are dealing with a large amount of data, preparing all the data in memory before operating on it will result in very high memory usage. Thus we will sacrifice a multi-cell approach of preparing all the data step-by-step, in order to make sure we don't load more stuff into memory at a time than we need.

In [ ]:
# branch buffer for filling our score trees
    # make our branch buffer
branch_buffer = {
    'charged_likelihood_combo': np.zeros(1,dtype=np.dtype('f8')),
    'clusterE_charged': np.zeros(1,dtype=np.dtype('f8')),
    'clusterE_neutral': np.zeros(1,dtype=np.dtype('f8'))
}

for dfile, trees in ur_trees.items():
    
    print (dfile)
    # prep the calo images
    print('\tPrepping calo images...')
    calo_images = {}
    for layer in layers:
        calo_images[layer] = mu.setupCells(trees['cluster'],layer)
    combined_images = np.concatenate(tuple([calo_images[layer] for layer in layers]), axis=1)

    # prep some extra combined input for energy regression
    print('\tPrepping extra inputs...')
    scaler_e = StandardScaler()
    scaler_cal = StandardScaler()
    scaler_eta = StandardScaler()
    
    e = trees['cluster'].array('clusterE')
    e_calib = trees['cluster'].array('cluster_ENG_CALIB_TOT')
    eta = trees['cluster'].array('clusterEta')
    
    # cleaning for e and e_calib (empirically needed for e_calib to remove values that are too large)
    epsilon = 1.0e-12
    e = np.where(e < epsilon, epsilon, e)
    e_calib = np.where(e_calib < epsilon, epsilon, e_calib)
    
    regression_cols = {}
    regression_cols['s_logE'] = scaler_e.fit_transform(np.log(e).reshape(-1,1))
    regression_cols['s_logECalib'] = scaler_cal.fit_transform(np.log(e_calib).reshape(-1,1))
    regression_cols['s_eta'] = scaler_eta.fit_transform(eta.reshape(-1,1))
    
    s_combined,scaler_combined = mu.standardCells(combined_images, layers)
    regression_input = np.column_stack((regression_cols['s_logE'], regression_cols['s_eta'],s_combined))

    # now find network scores
    print('\tCalculating network outputs...')
    model_scores = {}
    
    print('\t\tClassification... ', end='')
    # 1) flat networks
    for layer in flat_model_names:
        model = network_models[layer]
        model_scores[layer] = model.predict(calo_images[layer])[:,1] # [:,1] based on Max's code, this is input to combo network. Likelihood of being charged (vs. neutral)
    
    # 2) combo network
    name = 'combo'
    model = network_models[name]
    input_scores = np.column_stack([model_scores[layer] for layer in layers])
    model_scores[name] = model.predict(input_scores)[:,1] # likelihood of being charged pion (versus neutral pion)
    print('Done.')
    
    print('\t\tRegression... ', end='')
    # 3) energy regression networks
    name = 'e_charged'
    model = network_models[name]
    model_scores[name] = np.exp(scaler_cal.inverse_transform(model.predict(regression_input)))
    
    name = 'e_neutral'
    model = network_models[name]
    model_scores[name] = np.exp(scaler_cal.inverse_transform(model.predict(regression_input)))
    print('Done.')
    
    # Now we should save these scores to a new tree.
    f = rt.TFile(dfile, 'UPDATE')
    tree_name = 'ScoreTree'
    t = rt.TTree(tree_name, tree_name)
    
    print('Saving network scores to tree ' + tree_name + '... ',end='')    
    # --- Setup the branches using our buffer. This is a rather general/flexible code block. ---
    branches = {}
    for bname, val in branch_buffer.items():
        descriptor = bname
        bshape = val.shape
        if(bshape != (1,)):
            for i in range(len(bshape)):
                descriptor += '[' + str(bshape[i]) + ']'
        descriptor += '/'
        if(val.dtype == np.dtype('i2')): descriptor += 'S'
        elif(val.dtype == np.dtype('i4')): descriptor += 'I'
        elif(val.dtype == np.dtype('i8')): descriptor += 'L'
        elif(val.dtype == np.dtype('f4')): descriptor += 'F'
        elif(val.dtype == np.dtype('f8')): descriptor += 'D'
        else:
            print('Warning, setup issue for branch: ', key, '. Skipping.')
            continue
        branches[bname] = t.Branch(bname,val,descriptor)
    
    # Fill the model score tree, and save it to the local data file.
    nentries = model_scores['combo'].shape[0]
    for i in range(nentries):
        branch_buffer['charged_likelihood_combo'][0] = model_scores['combo'][i]
        branch_buffer['clusterE_charged'][0] = model_scores['e_charged'][i]
        branch_buffer['clusterE_neutral'][0] = model_scores['e_neutral'][i]
        t.Fill()
    
    t.Write(tree_name, rt.TObject.kOverwrite)
    f.Close()
    print('Done.')
    
tree_names['score'] = tree_name
ur_trees = {file:{tree_key:ur.open(file)[tree_name] for tree_key,tree_name in tree_names.items()} for file in data_filenames}

Now, we want to perform jet-clustering, where we'll use the regressed energies (and the classification score will tell us which regressed energy to use for each cluster).

In [ ]:
global_eta_cut = 0.3 # eta cut to be applied to all jets -- those we make and those we're given
global_truth_e_cut = 25. # GeV -- recall that jet energies are stored in keV!

# pavetext with info on our global eta cut
cut_info = '|#eta_{j}| <' + ' {val:.1f}'.format(val=global_eta_cut)
cut_pave = rt.TPaveText(0.7, 0.6, 0.9, 0.7, 'NDC')
cut_pave.SetFillColor(0)
cut_pave.SetBorderSize(0)
cut_pave.SetTextFont(42)
cut_pave.SetTextSize(0.04)
cut_pave.SetTextAlign(12)
cut_pave.AddText(cut_info)

In [ ]:
sys.path.append(fj_dir)
import fastjet as fj

# Jet clustering params
R = 0.4
pt_min = 0.0 # min jet pT (GeV) (appears to be 5.0 GeV for the other jets but we turn it off for now)
jet_def = fj.JetDefinition(fj.antikt_algorithm, R)
eta_max = global_eta_cut

#energy rescaling (all jet info is saved in MeV, cluster info is in GeV)
energy_scaling = 1.0e3

# classification threshold - scores below are considered neutral pion clusters, scores above are considered charged pion clusters
classification_threshold = 0.5

# branch buffer for our jet tree
branch_buffer = {
    'AntiKt4MLTopoJetsPt':rt.std.vector('float')(),
    'AntiKt4MLTopoJetsEta':rt.std.vector('float')(),
    'AntiKt4MLTopoJetsPhi':rt.std.vector('float')(),
    'AntiKt4MLTopoJetsE':rt.std.vector('float')()
}

for dfile, trees in ur_trees.items():
    
    # event info
    cluster_min = trees['event'].array('clusterCount') # naming convention is a bit funny! clusterCount gives event starting index in ClusterTree
    cluster_max = cluster_min + trees['event'].array('nCluster') - 1
    
    # cluster info (pre-existing) #TODO; we include reco cluster E info for debugging purposes only
    cluster_vec = np.column_stack(tuple(trees['cluster'].arrays(['clusterPt','clusterEta','clusterPhi','clusterE']).values()))
    
    # topo-cluster classifications for all of the clusters in this file
    cluster_classification = trees['score'].array('charged_likelihood_combo')
    
    # topo-cluster regressed energies for all clusters in this file (regressions assuming cluster comes from charged/neutral pion)
    cluster_energies = np.column_stack(tuple(trees['score'].arrays(['clusterE_charged','clusterE_neutral']).values()))
    
    # tree for saving jet info
    f = rt.TFile(dfile, 'UPDATE')
    tree_name = 'JetTree'
    t = rt.TTree(tree_name, tree_name)
    branches = {}
    for key,val in branch_buffer.items():
        branches[key] = t.Branch(key, val)
    
    vec_polar = rt.Math.PtEtaPhiEVector()    
    # loop over events
    nevents = trees['event'].numentries
    for i in range(nevents):
        
        # explicit list of cluster indices we're working with -- these are indices in ClusterTree, corresponding to event i
        cluster_idxs = np.linspace(cluster_min[i], cluster_max[i], cluster_max[i] - cluster_min[i] + 1, dtype=np.dtype('i8'))        
        # nCluster = cluster_idxs.shape[0]
                
        pseudojets = []
        for idx in cluster_idxs:
            energy = cluster_energies[idx,0] # swap in the regressed energy, start off assuming a charged pion
            if cluster_classification[idx] < classification_threshold: energy = cluster_energies[idx,1] # switch to neutral energy regression if dictated by classification
            
            # rescale the pT according to how we've changed the topo-cluster energy (don't touch eta, phi)
            pt = cluster_vec[idx,0] * energy / cluster_vec[idx,3]
            
            # create 4-vector representing the topo-cluster
            vec_polar.SetCoordinates(pt,cluster_vec[idx,1],cluster_vec[idx,2],energy)
            
            # make a fastjet PseudoJet object from this 4-vector, add it to the list that will be given to jet clustering
            pseudojets.append(fj.PseudoJet(vec_polar.Px(), vec_polar.Py(), vec_polar.Pz(), vec_polar.E())) # fastjet uses Cartesian as input
        
        # perform jet clustering
        jets = jet_def(pseudojets)
        
        # Apply optional minimum jet pT cut
        jet_pt = np.array([jet.pt() for jet in jets])
        jet_indices = np.linspace(0,len(jets)-1,len(jets),dtype=np.dtype('i8'))[jet_pt >= pt_min]
        jets = [jets[i] for i in jet_indices]
        
        # Apply optional maximum |eta| cut
        jet_eta = np.array([jet.eta() for jet in jets])
        jet_indices = np.linspace(0,len(jets)-1,len(jets),dtype=np.dtype('i8'))[np.abs(jet_eta) <= eta_max]
        jets = [jets[i] for i in jet_indices]

        njets = len(jets)
        # Save jet info to a TTree
        for key in branch_buffer.keys(): branch_buffer[key].clear()
            
        for j in range(njets):    
#             vec.SetCoordinates(jets[j].pt(), jets[j].eta(), jets[j].phi(), jets[j].e())
#             print(vec.E(), vec.P(), vec.M())
#             #print(jets[j].pt(), jets[j].e())
            branch_buffer['AntiKt4MLTopoJetsPt'].push_back(jets[j].pt() * energy_scaling)
            branch_buffer['AntiKt4MLTopoJetsEta'].push_back(jets[j].eta())
            branch_buffer['AntiKt4MLTopoJetsPhi'].push_back(jets[j].phi())
            branch_buffer['AntiKt4MLTopoJetsE'].push_back(jets[j].e() * energy_scaling)
        
        t.Fill()
    t.Write(tree_name, rt.TObject.kOverwrite)
    f.Close()

tree_names['jet'] = tree_name
ur_trees = {file:{tree_key:ur.open(file)[tree_name] for tree_key,tree_name in tree_names.items()} for file in data_filenames}

Now, we want to match the jets we just clustered with the truth jets, to see how well we've reconstructed things.

Here's how we will perform jet-matching:

- Get the list of all reco jets and truth jets for an event
- Loop through the truth jets
    - Find the closest reco jet within $\Delta R=0.3$, if it exists, and call it a match
        - If we fail to find a match, make a note of this
    - Take the matched reco jet off the list, so we don't match it a 2nd time
    
From matched jets, we will immediately compute $E_\text{reco}/E_\text{true}$ and histogram it (we will *not* be saving the matches directly to a file, for now).

We will perform this process for all the different reco jet definitions in the files, so that we can compare our method's energy resolution to the others'.

In [ ]:
# Returns pairs & lists of matched and unmatched indices. These indices are w.r.t.
# whatever is given as input -- if some jets have been dropped from lists (for not passing cuts)
# this will *not* be known to jet_matching(). (i.e. it will always work 
# internally with a set of sequential indices, with which it reports results.)

def jet_matching(reco_jets, truth_jets, max_distance = 0.3):
    ntruth = len(truth_jets['eta'])
    nreco = len(reco_jets['eta'])
    reco_indices = np.linspace(0, nreco, nreco + 1, dtype = np.dtype('i2'))
    
    #TLorentzVectors for computing deltaR
    vec1 = rt.Math.PtEtaPhiEVector()
    vec2 = rt.Math.PtEtaPhiEVector()

    matched_indices = []
    unmatched_truth = []
    unmatched_reco = []
    
    for i in range(ntruth):
        truth_eta = truth_jets['eta'][i]
        truth_phi = truth_jets['phi'][i]
        vec1.SetCoordinates(0.,truth_eta,truth_phi,0.)
        
        # get distances between this truth jet and all unmatched reco jets
        distances = np.zeros(nreco)
        for j in range(nreco):
            reco_idx = reco_indices[j]
            if(reco_idx < 0):
                distances[j] = -999.
                continue 
            vec2.SetCoordinates(0.,reco_jets['eta'][reco_idx],reco_jets['phi'][reco_idx],0.)
            distances[j] = rt.Math.VectorUtil.DeltaR(vec1,vec2)
            
        # now find the minimum distance, beware of negative values
        # see https://stackoverflow.com/a/37973409
        valid_idx = np.where(distances >= 0.)[0]
        
        if(len(valid_idx) == 0):
            unmatched_truth.append(i)
            continue
        
        match_idx = valid_idx[distances[valid_idx].argmin()]
        matched_indices.append((i, match_idx))
        reco_indices[match_idx] = -1.
    unmatched_reco = reco_indices[reco_indices > -1]
    
    return {'truth_reco':matched_indices, 'unmatched_truth':unmatched_truth, 'unmatched_reco':unmatched_reco}

**Work in progress:** We'll save the reco jet matching information to a new tree. Note that this will take a little bit of time to compute, but saving it like this will allow quicker access later on.

In [ ]:
# saving jet matching info to a new tree

# branch_buffer = {
#     'AntiKt4EMTopoJetsMatch':rt.std.vector('int')(),
#     'AntiKt4LCTopoJetsMatch':rt.std.vector('int')(),
#     'AntiKt4MLTopoJetsMatch':rt.std.vector('int')()
# }

# jet_definitions = {
#     'EM':('event', 'AntiKt4EMTopoJets'),
#     'LC':('event', 'AntiKt4LCTopoJets'),
#     'ML':('jet',   'AntiKt4MLTopoJets'),
#     'Truth':('event', 'AntiKt4TruthJets')
# }

# reco_jet_defs = ['EM','LC','ML']

# for dfile, tree in ur_trees.items():
        
# #     # tree for saving jet matching info
# #     f = rt.TFile(dfile, 'UPDATE')
# #     tree_name = 'JetMatchTree'
# #     t = rt.TTree(tree_name, tree_name)
# #     branches = {}
# #     for key,val in branch_buffer.items():
# #         branches[key] = t.Branch(key, val)
    
#     # Determine which jets pass our global eta cut
#     eta = {key:ur_trees[dfile][val[0]].array(val[1] + 'Eta') for key, val in jet_definitions.items()}
#     jet_indices = {key: x <= global_eta_cut for key,x in eta.items()}
    
#     # Apply our truth jet energy cut. Recall that jets have things stored in keV for now, whereas the cut is in GeV.
#     truth_energy = ur_trees[dfile][jet_definitions['Truth'][0]].array(jet_definitions['Truth'][1] + 'E')
#     jet_indices['Truth'] = jet_indices['Truth'] * (truth_energy >= 1.0e3 * global_truth_e_cut)
    
#     # We will also need phi info for performing the matching
#     phi = {key:ur_trees[dfile][val[0]].array(val[1] + 'Phi') for key, val in jet_definitions.items()}

#     nevents = ur_trees[dfile]['event'].numentries
#     for i in range(nevents):
        
#         # keep track of which indices are present and dropped
#         jet_tree_indices = {key:np.linspace(0,len(jet_indices[key])-1,len(jet_indices[key]),dtype=np.dtype('i2'))[jet_indices[key]] for key in jet_indices.keys()}



In [ ]:
R_max = 0.3

reco_jet_definitions = {
    'EM':('event', 'AntiKt4EMTopoJets'),
    'LC':('event', 'AntiKt4LCTopoJets'),
    'ML':('jet',   'AntiKt4MLTopoJets')
}

truth_jet_definition = 'AntiKt4TruthJets'
jet_energy_ratios = {x:[] for x in reco_jet_definitions.keys()}

for dfile in data_filenames:
    nevents = ur_trees[dfile]['event'].numentries
#     nevents = 10 #TODO: restricting range for debugging
    for i in range(nevents):
        
        # take only jets passing our global eta cut
        truth_eta = ur_trees[dfile]['event'].array(truth_jet_definition + 'Eta')[i]
        truth_indices = np.linspace(0,len(truth_eta)-1,len(truth_eta),dtype=np.dtype('i8'))[np.abs(truth_eta) <= global_eta_cut]
        
        reco_eta = {key: ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Eta')[i] for key, jet_def in reco_jet_definitions.items()}
        reco_indices = {key: np.linspace(0,len(eta)-1,len(eta),dtype=np.dtype('i8'))[np.abs(eta) <= global_eta_cut] for key, eta in reco_eta.items()}

        truth_jets = {'eta':ur_trees[dfile]['event'].array(truth_jet_definition + 'Eta')[i][truth_indices],'phi':ur_trees[dfile]['event'].array(truth_jet_definition + 'Phi')[i][truth_indices]}
        reco_jets = {key:{'eta':ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Eta')[i][reco_indices[key]],'phi':ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Phi')[i][reco_indices[key]]} for key, jet_def in reco_jet_definitions.items()}
        
        for key, jet_def in reco_jet_definitions.items():
            matching_results = jet_matching(reco_jets[key], truth_jets, max_distance = R_max)
            for match in matching_results['truth_reco']:
                e_truth = ur_trees[dfile]['event'].array(truth_jet_definition + 'E')[i,match[0]]
                e_reco = ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'E')[i,match[0]]
                e_ratio = e_reco / e_truth
                jet_energy_ratios[key].append(e_ratio)
                
jet_energy_ratios = {key:np.array(val) for key,val in jet_energy_ratios.items()}

In [ ]:
def SetColor(hist, color, alpha = 0.5):
    hist.SetLineColor(color)
    hist.SetFillColorAlpha(color, alpha)

In [ ]:
# now plot results

colors = {
    'EM':rt.kGreen,
    'LC':rt.kRed,
    'ML':rt.kBlue
}
min_ratio = 0.
max_ratio = 5.
nbins = 50

hists = {key: rt.TH1F(key + 'ratio', key + ';E_{reco}/E_{true};Count',nbins,min_ratio,max_ratio) for key in reco_jet_definitions.keys()}
for key in colors.keys():
    SetColor(hists[key],colors[key])
    for entry in jet_energy_ratios[key]:
        hists[key].Fill(entry)

nbins_factor = 1
mult_factor = 1.0e5
hists2 = {key: rt.TH1F(key + 'ratio', key + ';E_{reco}/E_{true};Count',int(nbins_factor * nbins),min_ratio,mult_factor * max_ratio) for key in reco_jet_definitions.keys()}
for key in colors.keys():
    SetColor(hists2[key],colors[key])
    for entry in jet_energy_ratios[key]:
        hists2[key].Fill(entry)
        
c = rt.TCanvas('c_ratio','c_ratio',800,600)
c2 = rt.TCanvas('c_ratio2','c_ratio2',800,600)

rt.gStyle.SetOptStat(0)
legend = rt.TLegend(0.7,0.7,0.9,0.85)
legend.SetBorderSize(0)

stack = rt.THStack('stack','Energy Ratio;E_{reco}/E_{truth};Count')
stack2 = rt.THStack('stack2','Energy Ratio (extended axis);E_{reco}/E_{truth};Count')

for key in hists.keys():
    stack.Add(hists[key])
    stack2.Add(hists2[key])
    legend.AddEntry(hists[key],key,'f')

c.cd()
stack.Draw('NOSTACK HIST')
legend.Draw()
cut_pave.Draw()

c2.cd()
stack2.Draw('NOSTACK HIST')
legend.Draw()
cut_pave.Draw()
c2.SetLogy()

c.Draw()
c2.Draw()

Some of the stuff we're seeing above looks weird. Many of the cluster energies are too low, but there's also a very large tail to the distribution.

Note that there seems to have been an issue with **units** that I have already accounted for: Based on the magnitude of their values, I think that the truth jets and existing reco jets (EM, LC) had their $p_T$ and energy values stored in keV, not GeV. I have adjusted the ML jets to store their info in keV too to match, and we will convert to GeV for all for plotting.

But even with this rescaling having been done, we see issues such as in the plot above.

## Verification

To get a better sense of what our data looks like, let's produce some kinematic plots for all flavors of jets. We'll see how the different jet definitions' kinematics compare, and if something is off with our ML jets.

For our reco jets, we will only be considering those that have been matched with truth jets.

In [ ]:
def DrawSet(hists, logx=False, logy=True, cut_pave = 0):
    canvas = rt.TCanvas(str(uuid.uuid4()), str(uuid.uuid4()), 800, 600)
    nx = 2
    l = len(hists.keys())
    ny = int(np.ceil(l / nx))
    canvas.Divide(nx, ny)
    for i, hist in enumerate(hists.values()):
        canvas.cd(i+1)
        hist.Draw('HIST')
        if(logx):
            rt.gPad.SetLogx()
            hist.GetXaxis().SetRangeUser(1.0e-0, hist.GetXaxis().GetBinUpEdge(hist.GetXaxis().GetLast()))
        if(logy): 
            rt.gPad.SetLogy()
            hist.SetMinimum(5.0e-1)
        else:
            hist.SetMinimum(0.)
        if(cut_pave != 0): cut_pave.Draw()
            
    return canvas

# plotting various jet energies divided by pT
jet_defs = {
    'Truth':('event','AntiKt4TruthJets'),
    'EM':('event', 'AntiKt4EMTopoJets'),
    'LC':('event', 'AntiKt4LCTopoJets'),
    'ML':('jet',   'AntiKt4MLTopoJets')
}

colors = {
    'Truth': rt.kOrange,
    'EM': rt.kGreen,
    'LC': rt.kRed,
    'ML': rt.kBlue
}

scale_factors = 0.001 # jet info seems to be in keV, we want to plot it all in GeV

energy_hists = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;Energy [GeV];Count', 100, 0., 500.) for key in jet_defs.keys()}
pt_hists     = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;p_{T} [GeV];Count', 30, 0., 150.) for key in jet_defs.keys()}
eta_hists    = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;#eta;Count', 50, -1., 1.) for key in jet_defs.keys()}
m_hists      = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;m [GeV];Count', 45, -250., 2000.) for key in jet_defs.keys()}
ep_hists     = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;Energy / p_{T};Count', 50, 0., 5.) for key in jet_defs.keys()}
n_hists      = {key:rt.TH1I(str(uuid.uuid4()), key + ' Jets;N_{jets};Count', 100, 0., 100) for key in jet_defs.keys()}

vec = rt.Math.PtEtaPhiEVector()
for dfile in data_filenames:
    for key, jet_def in jet_defs.items():
        
        eta         =                 ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Eta')
        n           = [len(x[np.abs(x) < global_eta_cut]) for x in eta]
        eta         = eta.flatten()
        jet_indices = np.linspace(0,len(eta)-1,len(eta),dtype=np.dtype('i8'))[np.abs(eta) <= global_eta_cut]
        eta         = eta[jet_indices]
        energy      = scale_factors * ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'E').flatten()[jet_indices]
        pt          = scale_factors * ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Pt').flatten()[jet_indices]
        ep          = energy / pt
#         n           = np.array([x.shape[0] for x in ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'E')],dtype=np.dtype('i2'))

#         print('Minimum pT for ' + key + ' = ' + str(np.min(pt)))
#         print('\tCorresponding energy =',energies[np.argmin(pt)])
        for i in range(len(n)):
            n_hists[key].Fill(n[i])
        for i in range(len(ep)):
            energy_hists[key].Fill(energy[i])
            pt_hists[key].Fill(pt[i])
            eta_hists[key].Fill(eta[i])
            ep_hists[key].Fill(ep[i])
            vec.SetCoordinates(pt[i],eta[i],0.,energy[i])
            m_hists[key].Fill(vec.M())

hist_lists = [energy_hists, pt_hists, eta_hists, ep_hists, m_hists, n_hists]
for key in jet_defs.keys():
    for hist_list in hist_lists:
        SetColor(hist_list[key],colors[key])            
            
rt.gStyle.SetOptStat(0)
canvases = []

for hist_list in hist_lists:
    logx = False
#     if(hist_list == energy_hists): logx=True
    c = DrawSet(hist_list, logx=logx, cut_pave = cut_pave)
    canvases.append(c)
    c.Draw()

It looks like our jet-finding, using the regressed energies, is producing a significant number of ML jets with negative mass. I think this is really just a result of the input energies being too low (so that $p^2 > E^2$). Besides the peak of the $m$ distribution being below zero for the ML jets, we also see this weird behavior exhibited in our plot of $E/p_T$.

### Verifying jet clustering

There are two things we can do to make sure that clustering is working as intended:
- Reproducing the EM jets.
- Flipping the classification of clusters to produce new ML jets.

#### 1) Reproducing EM jets

This code will look a lot like our jet clustering above, but we will be using the default reco energy. We'll save our new EM jets to a tree called `JetTree_EM`.

In [ ]:
sys.path.append(fj_dir)
import fastjet as fj

# Jet clustering params
R = 0.4
pt_min = 0.0 # min jet pT (GeV) (appears to be 5.0 GeV for the other jets but we turn it off for now)
jet_def = fj.JetDefinition(fj.antikt_algorithm, R)

#energy rescaling (all jet info is saved in keV, not GeV)
energy_scaling = 1.0e3

# branch buffer for our jet tree
branch_buffer = {
    'AntiKt4EMTopoJetsPt':rt.std.vector('float')(),
    'AntiKt4EMTopoJetsEta':rt.std.vector('float')(),
    'AntiKt4EMTopoJetsPhi':rt.std.vector('float')(),
    'AntiKt4EMTopoJetsE':rt.std.vector('float')()
}

for dfile, trees in ur_trees.items():
    
    # event info
    cluster_min = trees['event'].array('clusterCount')
    cluster_max = cluster_min + trees['event'].array('nCluster') - 1
    
    # cluster info (pre-existing)
    cluster_vec = np.column_stack(tuple(trees['cluster'].arrays(['clusterPt','clusterEta','clusterPhi','clusterE']).values()))
#     cluster_vec = np.column_stack(tuple(trees['cluster'].arrays(['clusterPt','clusterEta','clusterPhi','cluster_ENG_CALIB_TOT']).values()))


    # tree for saving jet info
    f = rt.TFile(dfile, 'UPDATE')
    tree_name = 'JetTree_EM'
    t = rt.TTree(tree_name, tree_name)
    
    branches = {}
    for key,val in branch_buffer.items():
        branches[key] = t.Branch(key, val)
    
    vec_polar = rt.Math.PtEtaPhiEVector()    
    # loop over events
    nevents = trees['event'].numentries
    for i in range(nevents):
        cluster_idxs = np.linspace(cluster_min[i], cluster_max[i], cluster_max[i] - cluster_min[i] + 1, dtype=np.dtype('i8'))        
        nCluster = cluster_idxs.shape[0]
                
        pseudojets = []
        for j, idx in enumerate(cluster_idxs):
            vec_polar.SetCoordinates(cluster_vec[idx,0],cluster_vec[idx,1],cluster_vec[idx,2],cluster_vec[idx,3])
            pseudojets.append(fj.PseudoJet(vec_polar.Px(), vec_polar.Py(), vec_polar.Pz(), vec_polar.E())) # fastjet uses Cartesian as input
        jets = jet_def(pseudojets) # perform jet clustering
        
        # Apply optional minimum jet pT cut
        jet_pt = np.array([jet.pt() for jet in jets])
        jet_indices = np.linspace(0,len(jets)-1,len(jets),dtype=np.dtype('i8'))[jet_pt >= pt_min]
        jets = [jets[i] for i in jet_indices]
        njets = len(jets)
        
        # TODO: save jet info to a TTree
        for key in branch_buffer.keys(): branch_buffer[key].clear()
            
        for j in range(njets):    
#             vec.SetCoordinates(jets[j].pt(), jets[j].eta(), jets[j].phi(), jets[j].e())
#             print(vec.E(), vec.P(), vec.M())
#             #print(jets[j].pt(), jets[j].e())
            branch_buffer['AntiKt4EMTopoJetsPt'].push_back(jets[j].pt() * energy_scaling)
            branch_buffer['AntiKt4EMTopoJetsEta'].push_back(jets[j].eta())
            branch_buffer['AntiKt4EMTopoJetsPhi'].push_back(jets[j].phi())
            branch_buffer['AntiKt4EMTopoJetsE'].push_back(jets[j].e() * energy_scaling)
        
        t.Fill()
    t.Write(tree_name, rt.TObject.kOverwrite)
    f.Close()

tree_names['jet_em'] = tree_name
ur_trees = {file:{tree_key:ur.open(file)[tree_name] for tree_key,tree_name in tree_names.items()} for file in data_filenames}

Now let's plot the kinematic distributions of our new EM jets and the old EM jets. Our hope is that they match. Note that we might expect to find some lower $p_T$ jets too, as it looks like a $p_T$ cut was applied to the original EM jets and we aren't necessarily applying one here.

In [ ]:
# plotting various jet energies divided by pT
jet_defs = {
    'EM':('event', 'AntiKt4EMTopoJets'),
    'EM2':('jet_em',   'AntiKt4EMTopoJets')
}

colors = {
    'EM': rt.kGreen,
    'EM2': rt.kViolet + 8
}

scale_factors = 0.001 # jet info seems to be in keV, we want to plot it all in GeV

energy_hists = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;Energy [GeV];Count', 100, 0., 100.) for key in jet_defs.keys()}
pt_hists     = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;p_{T} [GeV];Count', 100, 0., 100.) for key in jet_defs.keys()}
eta_hists    = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;#eta;Count', 100, -1., 1.) for key in jet_defs.keys()}
m_hists      = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;m [GeV];Count', 30, -10., 20.) for key in jet_defs.keys()}
ep_hists     = {key:rt.TH1F(str(uuid.uuid4()), key + ' Jets;Energy / p_{T};Count', 60, 0.8, 1.4) for key in jet_defs.keys()}
n_hists      = {key:rt.TH1I(str(uuid.uuid4()), key + ' Jets;N_{jets};Count', 20, 0, 20) for key in jet_defs.keys()}

vec = rt.Math.PtEtaPhiEVector()
for dfile in data_filenames:
    for key, jet_def in jet_defs.items():
        eta         =                 ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Eta')
        n           = [len(x[np.abs(x) < global_eta_cut]) for x in eta]
        eta         = eta.flatten()
        jet_indices = np.linspace(0,len(eta)-1,len(eta),dtype=np.dtype('i8'))[np.abs(eta) <= global_eta_cut]
        eta         = eta[jet_indices]
        energy      = scale_factors * ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'E').flatten()[jet_indices]
        pt          = scale_factors * ur_trees[dfile][jet_def[0]].array(jet_def[1] + 'Pt').flatten()[jet_indices]
        ep          = energy / pt
        
        for i in range(len(n)):
            n_hists[key].Fill(n[i])
        for i in range(len(ep)):
            energy_hists[key].Fill(energy[i])
            pt_hists[key].Fill(pt[i])
            eta_hists[key].Fill(eta[i])
            ep_hists[key].Fill(ep[i])
            vec.SetCoordinates(pt[i],eta[i],0.,energy[i])
            m_hists[key].Fill(vec.M())

hist_lists = [energy_hists, pt_hists, eta_hists, ep_hists, m_hists, n_hists]
for key in jet_defs.keys():
    for hist_list in hist_lists:
        SetColor(hist_list[key],colors[key])

legend = rt.TLegend(0.7,0.7,0.9,0.9)
legend.AddEntry(energy_hists['EM'],'EM','f')
legend.AddEntry(energy_hists['EM2'],'EM (new)','f')

rt.gStyle.SetOptStat(0)
canvases = []
stacks = []
for hist_list in hist_lists:
    c = rt.TCanvas(str(uuid.uuid4()), str(uuid.uuid4()), 800, 600)
    c.cd()
    c.SetLogy()
    stack = rt.THStack(str(uuid.uuid4()),'')
    stack.SetTitle(list(hist_list.values())[0].GetTitle())
    for key in hist_list.keys():
        stack.Add(hist_list[key])
    stack.Draw('NOSTACK HIST')
    legend.Draw()
    cut_pave.Draw()
    stack.GetXaxis().SetTitle(list(hist_list.values())[0].GetXaxis().GetTitle())
    stack.GetYaxis().SetTitle(list(hist_list.values())[0].GetYaxis().GetTitle())
    stack.SetMinimum(5.0e-1)
    canvases.append(c)
    stacks.append(stack)
    canvases[-1].Draw()

Hmmm... the energy and jet multiplicity distributions are *totally* different, the $p_T$ distributions are quite similar (with our new EM jets having some lower $p_T$ members as we anticipated). But that there's any big difference in the energy, eta and multiplicity distributions suggests that something odd is happening in jet clustering on our end. This warrants further investigation.

## Coding "Playground" below

As an experiment, let's try rescaling the energy ratios from earlier. Is the predicted energy maybe just off by some constant factor? It seems unlikely but it doesn't hurt to try and see if rescaling the histogram will give us a (sharp?) peak near unity.

In [ ]:
mean = np.mean(jet_energy_ratios['ML'])
print(mean)

Ok, I think the mean is thrown off by an outlier. So we can try to rescale the distribution so that the *median* or histogram *mode* is at 1.

In [ ]:
def DrawRescaledRatio(jet_energy_ratios, rescaling_factor, target_name):

    energy_ratio_rescaled = jet_energy_ratios['ML'] * rescaling_factor
    
    colors = {
        'EM':rt.kGreen,
        'LC':rt.kRed,
        'ML':rt.kBlue
    }
    min_ratio = 1.0e-3
    max_ratio = 1.0e1
    nbins = 10000

    hists = {key: rt.TH1F(key + 'ratio', key + ';E_{reco}/E_{true};Count',nbins,min_ratio,max_ratio) for key in ['EM','LC','ML']}
    for key in ['EM','LC']:
        SetColor(hists[key],colors[key])
        for entry in jet_energy_ratios[key]:
            hists[key].Fill(entry)

    # our rescaled ML jet histogram
    SetColor(hists['ML'],colors['ML'])
    for entry in energy_ratio_rescaled:
        hists['ML'].Fill(entry)

    legend = rt.TLegend(0.6,0.65,0.9,0.85)
    legend.SetBorderSize(0)

    stack = rt.THStack('stack2','Energy Ratio;E_{reco}/E_{truth};Count')

    for key, hist in hists.items():
        stack.Add(hist)
        name = key
        if(key == 'ML'): name = target_name
        legend.AddEntry(hist,name,'f')

    return stack, legend

In [ ]:
median = np.median(jet_energy_ratios['ML'])
rescaling_factor = 1. / median
stack, legend = DrawRescaledRatio(jet_energy_ratios, rescaling_factor, target_name='ML (rescaled with median)')

c = rt.TCanvas('c_ratio2','c_ratio2',800,600)
rt.gStyle.SetOptStat(0)
stack.Draw('NOSTACK HIST')
legend.Draw()
c.SetLogx()
c.SetLogy()
c.Draw()

Hmmm... rescaling the `ML` distribution to have a median of $1$ isn't quite cutting it. This suggests that there are a *lot* of events in the overflow bin.

Instead, let's try rescaling the distribution so that its peak is at $1$. Note that this is dependent on our choice of binning.

In [ ]:
nbins_tmp = 10000
hists['ML'] = rt.TH1F(key + 'ratio', 'ML;E_{reco}/E_{true};Count',nbins_tmp,min_ratio,max_ratio)
for entry in jet_energy_ratios['ML']:
    hists['ML'].Fill(entry)
binmax = hists['ML'].GetMaximumBin()
mode = hists['ML'].GetXaxis().GetBinCenter(binmax)
rescaling_factor = 1. / mode

stack, legend = DrawRescaledRatio(jet_energy_ratios, rescaling_factor, target_name='ML (rescaled with mode)')
c = rt.TCanvas('c_ratio3','c_ratio3',800,600)
rt.gStyle.SetOptStat(0)
stack.Draw('NOSTACK HIST')
legend.Draw()
c.Draw()

In [ ]:
c.SetLogx()
c.SetLogy()
c.Draw()

Perhaps unsurprisingly, there doesn't seem to be any obvious scaling that will improve things -- as we might expect, rescaling will stretch things out (and increase spread).